In [ ]:
#importing libraries
import time
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import os
import urllib.request
from PyPDF2 import PdfReader
import re
import pandas as pd
from openpyxl import load_workbook
from pathlib import Path

In [ ]:
def append_df_to_excel(df, excel_path):
    df_excel = pd.read_excel(excel_path)
    result = pd.concat([df_excel, df], ignore_index=True)
    result.to_excel(excel_path, index=False)
downloads = str(Path.home() / 'Downloads')
test_excel = str(Path.home() / 'Downloads/parser/test3.xlsx')
chromedriver_file = str(Path.home() / 'Downloads/parser/chromedriver.exe')

In [ ]:
def parser_1(mas, ids, declar, vvodi, metri, classned, kol, res):
    dictionary = {} #наш словарь
    amount = 1
#САЙТ
    if "тернет»:" in mas:
        index_fixed = mas.index('тернет»:')+1 #мб надо будет больше
        string = ''
        while mas[index_fixed]!="1.5":
            string += mas[index_fixed]
            index_fixed+=1
        dictionary['Сайт'] = string
    elif mas[mas.index('1.5')-1]!="сайта:":
        dictionary['Сайт'] = mas[mas.index('1.5')-1]
#НАЗВАНИЕ ПУНКТА
    if "9.2.6На" in mas:
        index_fixed = mas.index('9.2.6На')+10 #мб надо будет больше
        string = ''
        while mas[index_fixed]!="9.2.7" and mas[index_fixed]!="9.2.7Ок":
            string += mas[index_fixed]
            index_fixed+=1
        dictionary['Наименование населенного пункта'] = string
    elif mas[mas.index('9.2.6')-1]!="пункта:":
        dictionary['Наименование населенного пункта'] = mas[mas.index('9.2.6')-1]
#УЛИЦА
    if "9.2.10На" in mas:
        index_fixed = mas.index('9.2.10На')+6 #мб надо будет больше
        string = ''
        while(mas[index_fixed]!="9.2.11Дом:" and mas[index_fixed]!="9.2.11"):
            string += mas[index_fixed]
            index_fixed+=1
        dictionary['Наименование улицы'] = string
    elif mas[mas.index('9.2.10')-1]!="улицы:" and mas[mas.index('9.2.10')-1]!="цы:":
        if "9.2.11Дом:" in mas:
            if mas[mas.index('9.2.11Дом:')-1]!="цы:":
                dictionary['Наименование улицы'] = mas[mas.index('9.2.10')-1]
        else:
            dictionary['Наименование улицы'] = mas[mas.index('9.2.10')-1]
#КОЛИЧЕСТВО ДОМОВ
    if "9.1.2Обос" in mas:
        amount = int(mas2[mas2.index('9.1.2Обос')-1])
#КАДАСТРОВЫЙ НОМЕР
    if "12.3.2Пло" in mas:
        dictionary['Кадастровый номер земельного участка'] = mas[mas.index('12.3.2Пло')-1]
    elif mas[mas.index('12.3.2')-1]!="участка:":
        dictionary['Кадастровый номер земельного участка'] = mas[mas.index('12.3.2')-1]
#ПЛАНИРУЕМАЯ СТОИМОСТЬ
    if "ства18.1.1Пла" in mas:
        index_fixed = mas.index('ства18.1.1Пла')+8 #мб надо будет больше
        string = ''
        while(mas[index_fixed]!="18.1"):
            string += mas[index_fixed]
            index_fixed+=1
        dictionary['Планируемая стоимость строительства'] = string
    elif "18.1.1Пла" in mas:
        index_fixed = mas.index('18.1.1Пла')+8 #мб надо будет больше
        string = ''
        while(mas[index_fixed]!="18.1"):
            string += mas[index_fixed]
            index_fixed+=1
        dictionary['Планируемая стоимость строительства'] = string
    elif "18.1.1" in mas:
        index_fixed = mas.index('18.1.1')+4
        string = ''
        while(mas[index_fixed]!="19"):
            string += mas[index_fixed] + " "
            index_fixed+=1
        dictionary['Планируемая стоимость строительства'] = string
#ПЛАНИРУЕМАЯ ДАТА ПЕРЕДАЧИ
    if "17.2.2Пла" in mas:
        dictionary['Планируемая дата передачи застройщиком объекта долевого строительства'] = mas2[mas2.index('17.2.2Пла')+21]
#ЦИКЛ ДОМОВ
    square = []
    real = []
    quart = []
    floors = []
    if amount > res:
        amount = res
    for i in range(amount):
#ЭТАЖИ
        if "9.2.20Мак" in mas:
            floors.append(mas[mas.index('9.2.21Об')-1])
        elif mas[mas.index('9.2.20')-1]!="этажей:":
            floors.append(mas[mas.index('9.2.20')-1])
            mas[mas.index('9.2.20')] = 'void'
#ОБЩАЯ ПЛОЩАДЬ
        if "9.2.21Об" in mas:
            index_fixed = mas.index('9.2.21Об')+7 #мб надо будет больше
            string = ''
            while(mas[index_fixed]!="м2"):
                string += mas[index_fixed]
                index_fixed+=1
            square.append(string)
            mas[mas.index('9.2.21Об')] = 'void'
        elif mas[mas.index('9.2.21')-1]!="объекта:": #надо что-то придумать если несколько корпусов
            square.append(mas[mas.index('9.2.21')-2])
            mas[mas.index('9.2.21')] = 'void'
#ЭТАП РЕАЛИЗАЦИИ 
        if "17.1.2Пла" in mas:
            index_fixed = mas.index('ства17.1.1Этап')+10 #мб надо будет больше
            string = ''
            while(mas[index_fixed]!="17.1.2Пла"):
                string += mas[index_fixed]
                index_fixed+=1
            real.append(string)
            mas[mas.index('ства17.1.1Этап')] = 'void'
        else:
            index_fixed = mas.index('17.1.1')+5 #мб надо будет больше
            string = ''
            while(mas[index_fixed]!="17.1.2"):
                string += mas[index_fixed] + " "
                index_fixed+=1
            real.append(string)
            mas[mas.index('17.1.1')] = 'void'
#ПЛАНИРУЕМЫЙ КВАРТАЛ
        if "17.1.2Пла" in mas:
            index_fixed = mas.index('17.1.2Пла')+22 #мб надо будет больше
            string = ''
            while(mas[index_fixed]!="17.1" and mas[index_fixed]!="17.2"):
                string += mas[index_fixed]
                index_fixed+=1
            quart.append(string)
            mas[mas.index('17.1.2Пла')] = 'void'
        else:
            index_fixed = mas.index('17.1.2')+10
            string = ''
            while(mas[index_fixed]!="18" and mas[index_fixed]!="17.2" and mas[index_fixed]!="17.1"):
                string += mas[index_fixed] +' '
                index_fixed+=1
            quart.append(string)
            mas[mas.index('17.1.2')] = 'void'
    dictionary['Общая площадь объекта'] = square
    dictionary['Этап реализации'] = real
    dictionary['Планируемый квартал и год'] = quart
    dictionary['Максимальное кол-во этажей'] = floors
    return(dictionary)

In [ ]:
def parser_2(mas, ids, declar, vvodi, metri, classned, kol, res):
    dictionary = {} #наш словарь
#САЙТ
    if mas2[mas2.index('1.5')-1]!="«Интернет»:":
        dictionary['Сайт'] = mas2[mas2.index('1.5')-1]
    else:
        dictionary['Сайт'] = mas2[mas2.index('«Интернет»:')+1]
        if mas2[mas2.index('1.5')-1]=="«Интернет»:":
            dictionary['Сайт'] = ""
#НАЗВАНИЕ ПУНКТА
    if mas2[mas2.index('9.2.5')+1]=="Наименование":
        dictionary['Наименование населенного пункта'] = mas2[mas2.index('9.2.5')+4]
    elif mas2[mas2.index('9.2.7')-1]!="пункта:" and mas2[mas2.index('9.2.7')-1]!="пункте:":
        if mas2[mas2.index('9.2.7')-1]!="Город":
            dictionary['Наименование населенного пункта'] = mas2[mas2.index('9.2.7')-1]
        else:
            dictionary['Наименование населенного пункта'] = mas2[mas2.index('9.2.4')-1]
    else:
        index_fixed = mas2.index('9.2.5')+4 #мб надо будет больше
        string = ''
        first = mas2[index_fixed]
        while(mas2[index_fixed]!="9.2.6"):
            string += mas2[index_fixed] + " "
            index_fixed+=1
            if mas2[index_fixed] == first:
                break
        dictionary['Наименование населенного пункта'] = string
#УЛИЦА
    if mas2[mas2.index('9.2.11')-1]!="улицы:" and mas2[mas2.index('9.2.10')+1]!="Дом:":
        index_fixed = mas2.index('9.2.10')+3 #мб надо будет больше
        string = ''
        first = mas2[index_fixed]
        while(mas2[index_fixed]!="9.2.11" and mas2[index_fixed]!="дом"):
            string += mas2[index_fixed] + " "
            index_fixed+=1
            if mas2[index_fixed] == first:
                break
        dictionary['Наименование улицы'] = string
    elif mas2[mas2.index('9.2.9')+1]=="Наименование":
        index_fixed = mas2.index('9.2.9')+3 #мб надо будет больше
        string = ''
        first = mas2[index_fixed]
        while(mas2[index_fixed]!="9.2.10"):
            string += mas2[index_fixed] + " "
            index_fixed+=1
            if mas2[index_fixed] == first:
                break
        dictionary['Наименование улицы'] = string
    elif mas2[mas2.index('9.2.11')-1]!="Дом:" and mas2[mas2.index('9.2.11')+1]!="Дом:":
        dictionary['Наименование улицы'] = mas2[mas2.index('9.2.10')-1]
#КОЛИЧЕСТВО ДОМОВ
    amount = int(mas2[mas2.index('9.1.2')-1])
#КАДАСТРОВЫЙ НОМЕР
    if mas[mas2.index('12.3.2')-1]!="участка:":
        dictionary['Кадастровый номер земельного участка'] = mas2[mas2.index('12.3.2')-1]
#ПЛАНИРУЕМАЯ СТОИМОСТЬ
    index_fixed = mas2.index('18.1.1')+4
    string = ''
    first = mas2[index_fixed]
    while(mas2[index_fixed]!="18.1"):
        string += mas2[index_fixed] + " "
        index_fixed+=1
        if mas2[index_fixed] == first:
            break
    dictionary['Планируемая стоимость строительства'] = string
#ПЛАНИРУЕМАЯ ДАТА ПЕРЕДАЧИ
    if "17.2.2" in mas2:
        if mas[mas2.index('17.2.2')+8]!="18":
            dictionary['Планируемая дата передачи застройщиком объекта долевого строительства'] = mas2[mas2.index('17.2.2')+8]
#ЦИКЛ ДОМОВ
    square = []
    real = []
    quart = []
    floors = []
    if amount > res:
        amount = res
    for i in range(amount):
#ЭТАЖИ
        if mas2[mas2.index('9.2.21')-1]!="этажей:"  and mas2[mas2.index('9.2.21')-1]!="м2":
            floors.append(mas2[mas2.index('9.2.21')-1])
        else:
            floors.append(mas2[mas2.index('9.2.19')+4])
            mas2[mas2.index('9.2.19')] = 'void'   
#ОБЩАЯ ПЛОЩАДЬ
        if mas[mas2.index('9.2.21')-1]!="м2":
            index_fixed = mas2.index('9.2.21')+4
            string = ''
            while(mas2[index_fixed]!="м2"):
                string += mas2[index_fixed] + " "
                index_fixed+=1
            square.append(string)
            mas2[mas2.index('9.2.21')] = 'void'
        else:
            index_fixed = mas2.index('9.2.20')+4
            string = ''
            while(mas2[index_fixed]!="м2"):
                string += mas2[index_fixed] + " "
                index_fixed+=1
            square.append(string)
            mas2[mas2.index('9.2.20')] = 'void'            
#ЭТАП РЕАЛИЗАЦИИ
        index_fixed = mas2.index('17.1.1')+5 #мб надо будет больше
        string = ''
        first = mas2[index_fixed]
        while(mas2[index_fixed]!="17.1.2"):
            string += mas2[index_fixed] + " "
            index_fixed+=1
            if mas2[index_fixed] == first:
                break
        real.append(string)
        mas2[mas2.index('17.1.1')] = 'void'
#ПЛАНИРУЕМЫЙ КВАРТАЛ
        index_fixed = mas2.index('17.1.2')+10
        string = ''
        first = mas2[index_fixed]
        while(mas2[index_fixed]!="17.1" and mas2[index_fixed]!="18"):
            string += mas2[index_fixed] + " "
            index_fixed+=1
            if mas2[index_fixed] == first:
                break
        quart.append(string)
        mas2[mas2.index('17.1.2')] = 'void'
    dictionary['Общая площадь объекта'] = square
    dictionary['Этап реализации'] = real
    dictionary['Планируемый квартал и год'] = quart
    dictionary['Максимальное кол-во этажей'] = floors
    return(dictionary)

In [ ]:
link = "https://наш.дом.рф/сервисы/каталог-новостроек/объект/45321"
page = requests.get(link)
soup = BeautifulSoup(page.text, 'html.parser')
dec1 = soup.find_all('div')
link2 = "https://наш.дом.рф/сервисы/каталог-новостроек/список-объектов/список?objStatus=0&objectIds=45305%2C44265%2C18188&page=0&limit=20"
page2 = requests.get(link2)
soup2 = BeautifulSoup(page2.text, 'html.parser')
dec2 = soup2.find_all('span')
dec3 = soup.find_all('a')
i= 0
while i < len(dec3):
    try:
        #ssilki
        if len(str(dec3[i].attrs.values()).partition('styles__DownloadLink-sc-1fv64wt-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", ""))>1:
            ssilkid = str(dec3[i].attrs.values()).partition('styles__DownloadLink-sc-1fv64wt-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", "")       
        i+=1
    except TypeError:
        i+=1
i=0
while i < len(dec1):
    try:
        #vvod
        if len(str(dec1[i].attrs.values()).partition('styles__Container-sc-17wkiw0-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", ""))>1:
            vvodid = str(dec1[i].attrs.values()).partition('styles__Container-sc-17wkiw0-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", "")
        #dec
        if len(str(dec1[i].attrs.values()).partition('styles__Decl-kydjcf-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", ""))>1:
            decid = str(dec1[i].attrs.values()).partition('styles__Decl-kydjcf-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", "")
        #kvartir 
        if len(str(dec1[i].attrs.values()).partition('styles__Row-sc-1fyyfia-3')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", ""))>1:
            kvartirid = str(dec1[i].attrs.values()).partition('styles__Row-sc-1fyyfia-3')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", "")
        #vesegodomov
        if len(str(dec1[i].attrs.values()).partition('styles__CounterButtonWrapper-x75wld-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", ""))>1:
            vesegodomovid = str(dec1[i].attrs.values()).partition('styles__CounterButtonWrapper-x75wld-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", "")#['class'])
        #id
        #print(str(dec1[i].attrs.values()).partition('styles__Ellipsis-sc-1fw79ul-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", ""))
        i+=1
    except TypeError:
        i+=1
#ssilkid = str(dec3[4].attrs.values()).partition('styles__DownloadLink-sc-1fv64wt-0')[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "").replace("\n", "")       
i = 0
while i < len(dec2):
    try:
        if len(str(dec2[i].attrs.values()).partition('styles__Ellipsis-sc-1fw79ul-0')[2].partition('styles__Child-cx1nz2-0')[0].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "")) > 1:   
            idid1 = str(dec2[i].attrs.values()).partition('styles__Ellipsis-sc-1fw79ul-0')[2].partition('styles__Child-cx1nz2-0')[0].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "")[0:6]
        #if(len(str(dec2[i].attrs.values()).partition('styles__Ellipsis-sc-1fw79ul-0')[2].partition("', 'styles__Child-cx1nz2-0', 'styles__Primary-cx1nz2-1', '")[2].replace("'", "").replace(" ", "").replace(",", "").replace(")", "").replace("]", "")))<12:
        if len(str(dec2[i].attrs.values()).partition('styles__Ellipsis-sc-1fw79ul-0')[2].partition("', 'styles__Child-cx1nz2-0', 'styles__Primary-cx1nz2-1', '")[2].replace("'", "").replace(",", "").replace(")", "").replace("]", "").partition(" ")[0]) >1 : 
            idid2 = str(dec2[i].attrs.values()).partition('styles__Ellipsis-sc-1fw79ul-0')[2].partition("', 'styles__Child-cx1nz2-0', 'styles__Primary-cx1nz2-1', '")[2].replace("'", "").replace(",", "").replace(")", "").replace("]", "").partition(" ")[0]
        if len(str(dec2[i].attrs.values()).partition('styles__Ellipsis-sc-1fw79ul-0')[2].partition("', 'styles__Child-cx1nz2-0', 'styles__Primary-cx1nz2-1', '")[2].replace("'", "").replace(",", "").replace(")", "").replace("]", "")) > 1:
            idid3 = str(dec2[i].attrs.values()).partition('styles__Ellipsis-sc-1fw79ul-0')[2].partition("', 'styles__Child-cx1nz2-0', 'styles__Primary-cx1nz2-1', '")[2].replace("'", "").replace(",", "").replace(")", "").replace("]", "").partition(" ")[2]
        i+=1
    except TypeError:
        i+=1

In [ ]:
print(ssilkid, idid1, vvodid, decid, kvartirid, vesegodomovid, idid2)

In [ ]:
link = "https://наш.дом.рф/сервисы/каталог-новостроек/список-объектов/список?objStatus=0&objectIds=45305%2C44265%2C18188&page=0&limit=100"
page = requests.get(link)
soup = BeautifulSoup(page.text, 'html.parser')
par = soup.find_all('div',{'class':"styles__CounterButtonWrapper-x75wld-0 "+vesegodomovid})
vsegodomov = par[1].text[0:5]
#print(vsegodomov)

In [ ]:
id =[]
id = ['a' for i in range(int(vsegodomov))]
url=[]
url = ['a' for i in range(int(vsegodomov))]
declar = []
vvodi = []
metri = []
classned = []
kol = []
declar = ['a' for i in range(int(vsegodomov))]
vvodi = ['a' for i in range(int(vsegodomov))]
metri = ['a' for i in range(int(vsegodomov))]
classned = ['a' for i in range(int(vsegodomov))]
kol = ['a' for i in range(int(vsegodomov))]


In [ ]:
link = "https://наш.дом.рф/сервисы/каталог-новостроек/список-объектов/список?objStatus=0&objectIds=45305%2C44265%2C18188&page=0&limit=100"
page = requests.get(link)
soup = BeautifulSoup(page.text, 'html.parser')
par1 = soup.find_all('a',{'class':"pagination-item"})

realpage = par1[len(par1)-1].text

In [ ]:
options = Options()
options.add_argument('--disable-site-isolation-trials')
driver = webdriver.Chrome(chromedriver_file, options=options)
driver.implicitly_wait(10)
link = "https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA-%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA?objStatus=0&objectIds=45305%2C44265%2C18188&page=0&limit=100"
driver.get(link)
soup = BeautifulSoup(driver.page_source, 'html.parser')

par = soup.find_all('span',{'class':"styles__Ellipsis-sc-1fw79ul-0 "+ idid1 +" styles__Child-cx1nz2-0 styles__Primary-cx1nz2-1 "+idid2 +" " + idid3})
#print(par[0].text) 
#styles__Ellipsis-sc-1fw79ul-0 jYMONF styles__Child-cx1nz2-0 styles__Primary-cx1nz2-1 fnuOEA izGYuv
i = 0
while i<len(par):
    id[i] = par[i].text
    i+=1
#print(par)
j=1
#print(realpage)
#for item in par:
while j<int(realpage):
    driver.get("https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA-%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA?objStatus=0&page="+str(j)+"&limit=100")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')    
    par = soup.find_all('span',{'class':"styles__Ellipsis-sc-1fw79ul-0 "+idid1+" styles__Child-cx1nz2-0 styles__Primary-cx1nz2-1 "+idid2+" "+idid3})
    i=0
    while i<len(par):
        m =j*100+i
        id[m] = par[i].text
        i+=1
    j+=1
#print(id)

In [ ]:
#заносим айдишки из экселя в массив values
workbook = load_workbook(filename=test_excel)
sheet = workbook.active
values=[] #массив с айди из экселя
p=0
for row in sheet.values:
    if p==0:
        p+=1
        continue
    values.append(row[7])
i = 0
#print(len(id))
while i<len(id):
    if int(id[i]) not in values:
        link = "https://xn--80az8a.xn--d1aqf.xn--p1ai/сервисы/каталог-новостроек/объект/"+id[i]
        page = requests.get(link)
        soup = BeautifulSoup(page.text, 'html.parser')
        par = soup.find_all('a',{'class':"styles__DownloadLink-sc-1fv64wt-0 "+ssilkid})
        for item in par:
            url[i]=item['href']
        #print(item['href'])
        dec = soup.find_all('div',{'class':"styles__Decl-kydjcf-0 "+decid})
        vvod = soup.find_all('div',{'class':"styles__Container-sc-17wkiw0-0 "+vvodid})
        kvartir = soup.find_all('div',{'class':"styles__Row-sc-1fyyfia-3 "+kvartirid})
        metr= soup.find_all('div',{'class':"styles__Container-sc-17wkiw0-0 "+vvodid})
        try:
            declar[i] = dec[0].text.partition(' ')[0]#Проектная декларация
            #print(id[i])
            vvodi[i] = vvod[0].text.partition('Ввод в эксплуатацию')[2].partition('Выдача ключей')[0] #Ввод в эксплуатацию
            classned[i] = kvartir[0].text.partition('Класс недвижимости')[2] #Класс недвижимости
            kol[i] = kvartir[5].text.partition('Количество квартир')[2] #Количество квартир
            metri[i] = metr[0].text.partition('м²')[2].partition('Распроданность квартир')[0] #Средняя цена за 1 м²
        #print(" Проектная декларация" + dec[0].text.partition(' ')[0])
        #print("Ввод в эксплуатацию " + vvod[0].text.partition('Ввод в эксплуатацию')[2].partition('Выдача ключей')[0])
        #print("Класс недвижимости" + kvartir[0].text.partition('Класс недвижимости')[2])
        #print("Количество квартир " + kvartir[5].text.partition('Количество квартир')[2])
        #print("ср цена " + metr[0].text.partition('м²')[2].partition('Распроданность квартир')[0]+"\n"+"VSE")
         #print(i)
        except IndexError:
            i+=1
    i+=1
#print(url)

In [ ]:
i = 0
filename_last = []
while i<len(url):
    try:
        filename =url[i].partition('=')[2]
        id_mas = []
        declar_mas = []
        vvodi_mas = []
        metri_mas = []
        classned_mas = []
        kol_mas = []
        for j in range(len(url)):
            filename_j =url[j].partition('=')[2]
            if filename_j[-13:-4]==filename[-13:-4]:
                id_mas.append(id[j])
                declar_mas.append(declar[j])
                vvodi_mas.append(vvodi[j])
                metri_mas.append(metri[j])
                classned_mas.append(classned[j])
                kol_mas.append(kol[j])
        if filename[-13:-4] not in filename_last:
            urllib.request.urlretrieve(url[i], downloads+filename )
            reader = PdfReader(downloads+filename) #открываем файл
            mas=[]
            mas2=[]
            p=0
            for page in reader.pages:
                mas += [str(s) for s in page.extract_text().split()] #читаем всю пдфку
            for item in mas:
                if '\u200b' in item:
                    item = item.replace('\u200b','') #убираем непонятную хрень
                    p=1
                mas2.append(item)
        #print(mas2)
            if p==0:
                try:
                    restriction = len(id_mas)
                    dictionary = parser_1(mas2, id[i], declar[i], vvodi[i], metri[i], classned[i], kol[i], restriction) #первый тип парсера
                    #id дома
                    dictionary['ID дома'] = id_mas
                    #Проектная декларация
                    dictionary['Проектная декларация'] = declar_mas
                    #Ввод в эксплуатацию
                    dictionary['Ввод в эксплуатацию'] = vvodi_mas
                    # Средняя цена за 1 м²
                    dictionary['Средняя цена за 1 м²'] = metri_mas
                    #Класс недвижимости
                    dictionary['Класс недвижимости'] = classned_mas
                    #Кол-во квартир 
                    dictionary['Количество квартир'] = kol_mas
                    df = pd.DataFrame(dictionary)
                    df.style.hide_columns()
                    append_df_to_excel(df, test_excel)
                except(IndexError, ValueError):
                    print(filename)
            if p==1:
                try:
                    restriction = len(id_mas)
                    dictionary = parser_2(mas2, id[i], declar[i], vvodi[i], metri[i], classned[i], kol[i], restriction) #второй тип парсера
                    #id дома
                    dictionary['ID дома'] = id_mas
                    #Проектная декларация
                    dictionary['Проектная декларация'] = declar_mas
                    #Ввод в эксплуатацию
                    dictionary['Ввод в эксплуатацию'] = vvodi_mas
                    # Средняя цена за 1 м²
                    dictionary['Средняя цена за 1 м²'] = metri_mas
                    #Класс недвижимости
                    dictionary['Класс недвижимости'] = classned_mas
                    #Кол-во квартир 
                    dictionary['Количество квартир'] = kol_mas
                    #print(dictionary)
                    df = pd.DataFrame(dictionary)
                    #print(df)
                    df.style.hide_columns()
                    append_df_to_excel(df, test_excel)
                except(IndexError, ValueError):
                    print(filename)
        #print(filename)
        #filename check
            os.remove(downloads+filename)
            filename_last.append(filename[-13:-4])
    #print(filename_last)
        i+=1
    except ValueError:
        i+=1